In [12]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from datetime import datetime
import os
import wandb
from pathlib import Path

# 본 과제 제출자는 현재 우분투 도커 환경에서 작업중이므로 다음과 같이 경로 설정
BASE_PATH="/home/Deep-Learning-study"
import sys
sys.path.append(BASE_PATH)

CURRENT_FILE_PATH = os.getcwd()
CHECKPOINT_FILE_PATH = os.path.join(CURRENT_FILE_PATH, "checkpoints")

if not os.path.isdir(CHECKPOINT_FILE_PATH):
  os.makedirs(os.path.join(CURRENT_FILE_PATH, "checkpoints"))

In [13]:
from _01_code._08_fcn_best_practice.c_trainer import ClassificationTrainer
from _01_code._15_lstm_and_its_application.f_arg_parser import get_parser
from _01_code._15_lstm_and_its_application.g_crypto_currency_regression_train_lstm import get_btc_krw_data

In [14]:
def get_model():
    class MyModel(nn.Module):
        def __init__(self, n_input, n_output):
            super().__init__()
            
            # 메인 LSTM 레이어
            self.lstm = nn.LSTM(
                input_size=n_input,
                hidden_size=1024,  # hidden size 증가
                num_layers=3,      # 3개의 layer
                dropout=0.1,       # dropout 추가
                batch_first=True,
                bidirectional=True # 양방향 LSTM
            )
            
            # 분류를 위한 FC 레이어
            self.fc_layers = nn.Sequential(
                nn.LayerNorm(2048),  # bidirectional이므로 hidden_size * 2
                nn.Linear(2048, 512),
                nn.GELU(),
                nn.Dropout(0.1),
                
                nn.LayerNorm(512),
                nn.Linear(512, 128),
                nn.GELU(),
                nn.Dropout(0.1),
                
                nn.LayerNorm(128),
                nn.Linear(128, n_output),  # n_output=2 for binary classification
            )

        def forward(self, x):
            self.lstm.flatten_parameters()  # CUDA 성능 최적화
            x, _ = self.lstm(x)
            x = x[:, -1, :]  # 마지막 시퀀스의 출력만 사용
            x = self.fc_layers(x)
            return x  # CrossEntropyLoss를 사용할 것이므로 softmax는 여기서 적용하지 않음

    my_model = MyModel(n_input=5, n_output=2)
    return my_model

# Args 클래스도 classification task에 맞게 수정
class Args:
    def __init__(self):
        self.wandb = True
        self.batch_size = 32       # classification은 regression보다 큰 배치 사이즈가 효과적일 수 있음
        self.epochs = 300
        self.learning_rate = 1e-3  # classification은 보통 더 큰 학습률 사용
        self.weight_decay = 1e-4
        self.validation_intervals = 1
        self.early_stop_patience = 30
        self.early_stop_delta = 1e-4

In [15]:
def main(args):
  run_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

  config = {
    'epochs': args.epochs,
    'batch_size': args.batch_size,
    'validation_intervals': args.validation_intervals,
    'learning_rate': args.learning_rate,
    'early_stop_patience': args.early_stop_patience,
    'early_stop_delta': args.early_stop_delta,
    'weight_decay': args.weight_decay
  }

  project_name = "lstm_classification_btc_krw"
  wandb.init(
    mode="online" if args.wandb else "disabled",
    project=project_name,
    notes="btc_krw experiment with lstm",
    tags=["lstm", "classification", "btc_krw"],
    name=run_time_str,
    config=config
  )
  print(args)
  print(wandb.config)

  train_data_loader, validation_data_loader, _ = get_btc_krw_data(is_regression=False)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print(f"Training on device {device}.")

  model = get_model()
  model.to(device)

  optimizer = optim.Adam(model.parameters(), lr=wandb.config.learning_rate, weight_decay=wandb.config.weight_decay)

  classification_trainer = ClassificationTrainer(
    project_name, model, optimizer, train_data_loader, validation_data_loader, None,
    run_time_str, wandb, device, CHECKPOINT_FILE_PATH
  )
  classification_trainer.train_loop()

  wandb.finish()

In [16]:
if __name__ == "__main__":
    import sys
    if 'ipykernel' in sys.modules:  # Jupyter Notebook에서 실행 중인지 확인
        # Jupyter에서 실행할 때는 기본값 사용
        args = Args()
    else:
        # 일반 Python 스크립트로 실행할 때는 argparse 사용
        parser = get_parser()
        args = parser.parse_args()
    
    main(args)

{'epochs': 300, 'batch_size': 32, 'validation_intervals': 1, 'learning_rate': 0.001, 'early_stop_patience': 30, 'early_stop_delta': 0.0001, 'weight_decay': 0.0001}
Training on device cuda:0.
[Epoch   1] T_loss: 0.72452, T_accuracy: 49.6674 | V_loss: 0.68978, V_accuracy: 56.0000 | Early stopping is stated! | T_time: 00:00:05, T_speed: 0.200
[Epoch   2] T_loss: 0.69513, T_accuracy: 52.6330 | V_loss: 0.68879, V_accuracy: 56.0000 | V_loss decreased (0.68978 --> 0.68879). Saving model... | T_time: 00:00:10, T_speed: 0.200
[Epoch   3] T_loss: 0.69390, T_accuracy: 51.4412 | V_loss: 0.69987, V_accuracy: 56.0000 | Early stopping counter: 1 out of 30 | T_time: 00:00:18, T_speed: 0.167
[Epoch   4] T_loss: 0.69859, T_accuracy: 50.5820 | V_loss: 0.69315, V_accuracy: 44.0000 | Early stopping counter: 2 out of 30 | T_time: 00:00:23, T_speed: 0.174
[Epoch   5] T_loss: 0.69406, T_accuracy: 51.3581 | V_loss: 0.68454, V_accuracy: 56.0000 | V_loss decreased (0.68879 --> 0.68454). Saving model... | T_time:

Epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Training accuracy (%),▁█▅▃▅▅▅▇▅▄▅▆▇▅▃▇▆▆▆▄▅▇▆▇▆█▇██▇██▇████▇██
Training loss,█▂▂▃▂▂▂▂▁▁▁▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Training speed (epochs/sec.),██▂▃▄▁▂▂▂▃▃▃▃▂▂▂▃▃▃▃▃▃▃▄▃▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
Validation accuracy (%),███▁█▁▁██████▁███▁██████████████████████
Validation loss,▄▃▅▄▃█▅▅▃▅▆█▁▆▃▅▄▅▄▃▄▄▄▅▁▃▅▃▂▁▄▅▃▆▂▃▄▄▆▂
Epoch,44
Training accuracy (%),52.77162
Training loss,0.69238
Training speed (epochs/sec.),0.17959
Validation accuracy (%),56
